## KMeans Clustering

In [ ]:
%pyspark
raw_data = sc.textFile('file:/tmp/iris.csv')
raw_data.take(3)

header = raw_data.first()
skip_data = raw_data.filter(lambda line : line != header)
skip_data.take(3)

import numpy 
parsedData = skip_data.map(lambda e: numpy.array(e.split(',')[0:4]) )
parsedData.take(3)

from pyspark.mllib.clustering import KMeans
clusters = KMeans.train(parsedData,3, maxIterations=10,runs=30, initializationMode="random")

iris1 = parsedData.first()
iris1
clusters.predict(iris1)

prediction = clusters.predict(parsedData)
prediction.collect()